<a href="https://colab.research.google.com/github/ranafaraz/financial_sentiment_analysis/blob/main/TFIDF_Loughran_McDonald_Word2Vec_Blending_ML_Classifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [2]:
import os
import psutil
import platform
import scipy.sparse as sp
from scipy.sparse import csr_matrix, hstack
# import joblib
# from itertools import permutations
from itertools import combinations, chain
import numpy as np
import pandas as pd
import re
import pytz # timezone
from datetime import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from imblearn.over_sampling import SMOTE
from tqdm import tqdm
import multiprocessing
from multiprocessing import Process, Queue


from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, cohen_kappa_score
# from sklearn.base import clone
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier
# from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install num2words
from num2words import num2words



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=c9ff8311a04f339c437ae1333a4dcc924ee648bd524b9f7f15f74a473b90a464
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
# Load the dataset
df = pd.read_csv("/content/Drive/MyDrive/python/data.csv")
# custom_stopwords_df = pd.read_csv("/content/Drive/MyDrive/python/financial_sentiment_analysis/custom_stopwords.csv")  # StopWord List


A Python module that will load the Loughran-McDonald Master and its components (and optionally separate sentiment dictionaries).

But this technique doesn't worked well in our case. It only added few new features.

In [4]:

"""
Routine to load master dicitonary
Version for LM 2020 Master Dictionary

Bill McDonald
Date: 201510 Updated: 202201
"""

import datetime as dt
import sys


def load_masterdictionary(file_path, print_flag=False, f_log=None, get_other=False):
    start_local = dt.datetime.now()
    # Setup dictionaries
    _master_dictionary = {}
    _sentiment_categories = ['negative', 'positive', 'uncertainty', 'litigious',
                             'strong_modal', 'weak_modal', 'constraining']
    _sentiment_dictionaries = dict()
    for sentiment in _sentiment_categories:
        _sentiment_dictionaries[sentiment] = dict()

    # Load slightly modified common stopwords.
    # Dropped from traditional: A, I, S, T, DON, WILL, AGAINST
    # Added: AMONG
    _stopwords = ['ME', 'MY', 'MYSELF', 'WE', 'OUR', 'OURS', 'OURSELVES', 'YOU', 'YOUR', 'YOURS',
                  'YOURSELF', 'YOURSELVES', 'HE', 'HIM', 'HIS', 'HIMSELF', 'SHE', 'HER', 'HERS', 'HERSELF',
                  'IT', 'ITS', 'ITSELF', 'THEY', 'THEM', 'THEIR', 'THEIRS', 'THEMSELVES', 'WHAT', 'WHICH',
                  'WHO', 'WHOM', 'THIS', 'THAT', 'THESE', 'THOSE', 'AM', 'IS', 'ARE', 'WAS', 'WERE', 'BE',
                  'BEEN', 'BEING', 'HAVE', 'HAS', 'HAD', 'HAVING', 'DO', 'DOES', 'DID', 'DOING', 'AN',
                  'THE', 'AND', 'BUT', 'IF', 'OR', 'BECAUSE', 'AS', 'UNTIL', 'WHILE', 'OF', 'AT', 'BY',
                  'FOR', 'WITH', 'ABOUT', 'BETWEEN', 'INTO', 'THROUGH', 'DURING', 'BEFORE',
                  'AFTER', 'ABOVE', 'BELOW', 'TO', 'FROM', 'UP', 'DOWN', 'IN', 'OUT', 'ON', 'OFF', 'OVER',
                  'UNDER', 'AGAIN', 'FURTHER', 'THEN', 'ONCE', 'HERE', 'THERE', 'WHEN', 'WHERE', 'WHY',
                  'HOW', 'ALL', 'ANY', 'BOTH', 'EACH', 'FEW', 'MORE', 'MOST', 'OTHER', 'SOME', 'SUCH',
                  'NO', 'NOR', 'NOT', 'ONLY', 'OWN', 'SAME', 'SO', 'THAN', 'TOO', 'VERY', 'CAN',
                  'JUST', 'SHOULD', 'NOW', 'AMONG']

    # Loop thru words and load dictionaries
    with open(file_path) as f:
        _total_documents = 0
        _md_header = f.readline()  # Consume header line
        print()
        for line in f:
            cols = line.rstrip('\n').split(',')
            word = cols[0]
            _master_dictionary[word] = MasterDictionary(cols, _stopwords)
            for sentiment in _sentiment_categories:
                if getattr(_master_dictionary[word], sentiment):
                    _sentiment_dictionaries[sentiment][word] = 0
            _total_documents += _master_dictionary[cols[0]].doc_count
            if len(_master_dictionary) % 5000 == 0 and print_flag:
                print(f'\r ...Loading Master Dictionary {len(_master_dictionary):,}', end='', flush=True)

    if print_flag:
        print('\r', end='')  # clear line
        print(f'\nMaster Dictionary loaded from file:\n  {file_path}\n')
        print(f'  master_dictionary has {len(_master_dictionary):,} words.\n')

    if f_log:
        try:
            f_log.write('\n\n  FUNCTION: load_masterdictionary' +
                        '(file_path, print_flag, f_log, get_other)\n')
            f_log.write(f'\n    file_path  = {file_path}')
            f_log.write(f'\n    print_flag = {print_flag}')
            f_log.write(f'\n    f_log      = {f_log.name}')
            f_log.write(f'\n    get_other  = {get_other}')
            f_log.write(f'\n\n    {len(_master_dictionary):,} words loaded in master_dictionary.\n')
            f_log.write(f'\n    Sentiment:')
            for sentiment in _sentiment_categories:
                f_log.write(f'\n      {sentiment:13}: {len(_sentiment_dictionaries[sentiment]):8,}')
            f_log.write(f'\n\n  END FUNCTION: load_masterdictionary: {(dt.datetime.now()-start_local)}')
        except Exception as e:
            print('Log file in load_masterdictionary is not available for writing')
            print(f'Error = {e}')

    if get_other:
        return _master_dictionary, _md_header, _sentiment_categories, _sentiment_dictionaries, _stopwords, _total_documents
    else:
        return _master_dictionary


class MasterDictionary:
    def __init__(self, cols, _stopwords):
        for ptr, col in enumerate(cols):
            if col == '':
                cols[ptr] = '0'
        try:
            self.word = cols[0].upper()
            self.sequence_number = int(cols[1])
            self.word_count = int(cols[2])
            self.word_proportion = float(cols[3])
            self.average_proportion = float(cols[4])
            self.std_dev_prop = float(cols[5])
            self.doc_count = int(cols[6])
            self.negative = int(cols[7])
            self.positive = int(cols[8])
            self.uncertainty = int(cols[9])
            self.litigious = int(cols[10])
            self.strong_modal = int(cols[11])
            self.weak_modal = int(cols[12])
            self.constraining = int(cols[13])
            self.syllables = int(cols[14])
            self.source = cols[15]
            if self.word in _stopwords:
                self.stopword = True
            else:
                self.stopword = False
        except:
            print('ERROR in class MasterDictionary')
            print(f'word = {cols[0]} : seqnum = {cols[1]}')
            quit()
        return


In [5]:
# Calling the method to load Loughran-McDonald Master Dictionary
if __name__ == '__main__':
    start = dt.datetime.now()
    print(f'\n\n{start.strftime("%c")}\nPROGRAM NAME: {sys.argv[0]}\n')
    f_log = open('/content/Drive/MyDrive/python/financial_sentiment_analysis/Load_MD_Logfile.txt', 'w')
    md = (r"/content/Drive/MyDrive/python/financial_sentiment_analysis/Loughran-McDonald_MasterDictionary_1993-2021.csv")
    master_dictionary, md_header, sentiment_categories, sentiment_dictionaries, stopwords, total_documents = \
        load_masterdictionary(md, True, f_log, True)
    print(f'\n\nRuntime: {(dt.datetime.now()-start)}')
    print(f'\nNormal termination.\n{dt.datetime.now().strftime("%c")}\n')



Sat Sep 30 17:07:15 2023
PROGRAM NAME: /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py


 ...Loading Master Dictionary 85,000
Master Dictionary loaded from file:
  /content/Drive/MyDrive/python/financial_sentiment_analysis/Loughran-McDonald_MasterDictionary_1993-2021.csv

  master_dictionary has 86,531 words.



Runtime: 0:00:03.038235

Normal termination.
Sat Sep 30 17:07:18 2023



Subword tokenization using libraries like SentencePiece or Byte Pair Encoding (BPE) is useful when you want to tokenize text into subword units. This can be particularly helpful for languages with complex morphology or when dealing with out-of-vocabulary words. Here's how you can use these libraries:

Using SentencePiece:
SentencePiece is a popular subword tokenization library developed by Google. It's used for a wide range of natural language processing tasks.

But this technique doesn't worked in our case. Accuracy and number of features fell dramatically, as soon as we applied it.

In [5]:
# Train a SentencePiece model for Tokenization
!pip install sentencepiece
import sentencepiece as spm
spm.SentencePieceTrainer.train(input=(r"/content/Drive/MyDrive/python/data.csv"), model_prefix='mymodel', vocab_size=10000) # vocab_size=5000
sp = spm.SentencePieceProcessor(model_file='mymodel.model')

The following cell is dedicated for preprocessing and tokenization processes.

In [6]:

# Custom Tokenization function
def custom_tokenize(text):
    # Regular expressions to tokenize financial-specific terms and symbols
    # Example: Tokenize stock tickers like AAPL as single tokens
    text = re.sub(r'([A-Z]+)', r'\1 ', text)  # Add space after uppercase letters

    # Example: Tokenize currency symbols (e.g., $, €) and percentages
    text = re.sub(r'([$€%])', r' \1 ', text)  # Convert currency symbols and percentages to standardized forms

    # Tokenize numbers and symbols separately, and keep decimal numbers intact
    text = re.sub(r'(\d+\.\d+|\d+)', r' \1 ', text)  # Separate numbers and decimals
    text = re.sub(r'([$€%])', r' \1 ', text)  # Separate currency symbols and percentages

    # Tokenize the text into words
    words = text.split()

    # Initialize lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Apply lemmatization and lowercase, filter out single letters and common stopwords
    tokens = [lemmatizer.lemmatize(word.lower()) for word in words if len(word) > 1 and word.lower() not in stopwords.words('english')]

    # Filter out single characters and symbols
    tokens = [word for word in words if len(word) > 1 or re.match(r'[$€%]', word)]

    return tokens

# Function to convert numbers to text representations
def convert_numbers_to_text(tokens):
    converted_tokens = []
    for token in tokens:
        if token.replace('.', '', 1).isdigit():  # Check if it's a number
            converted_token = num2words(token)  # Convert number to text (e.g., "$1.50" to "one point five zero dollars")
            converted_tokens.append(converted_token)
        else:
            converted_tokens.append(token)
    return converted_tokens

# Data Preprocessing
def preprocess_text(text):

    # Define a regular expression pattern to remove URLs and punctuation marks
    # pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|[.,?!:;"`(){}\[\]<>]'
    # Remove URLs and punctuation marks
    # cleaned_text = re.sub(pattern, '', text)

    # Tokenization using custom tokenization function
    # tokens = custom_tokenize(cleaned_text)
    tokens = word_tokenize(text)
    # tokens = sp.encode_as_pieces(text) # Tokenizing Using SentencePiece
    # print("Tokens:", tokens)

    # Store the count of tokens before removing stopwords
    # tokens_before_removal = tokens.copy()

    # Remove stopwords
    # stop_words = set(stopwords.words('english'))
    # removed_words = [word for word in tokens_before_removal if word in stop_words]
    # tokens = [word for word in tokens if word not in stop_words]

    # Calculate how many stopwords were removed
    # removed_stopwords_count = len(tokens_before_removal) - len(tokens)
    # print(f"Removed {removed_stopwords_count} stopwords.")
    # print(f"Removed words: {removed_words}")

    # stemmer = PorterStemmer()
    # words = [stemmer.stem(word) for word in tokens]

    # Numerical handling (convert numbers to text representations)
    tokens = convert_numbers_to_text(tokens)

    # Join the words back to form preprocessed text
    preprocessed_text = ' '.join(tokens)

    # Print the tokens for debugging
    # first_100_tokens = tokens[:100]
    # print(first_100_tokens)

    # Return the text and sentiment scores
    return preprocessed_text

# df['Sentence'].apply(preprocess_text)

In [7]:
# Methods

# Function to convert bytes to MB or GB
def bytes_to_mb_or_gb(byte_value):
    if byte_value >= 1024**3:  # GB
        return f"{byte_value / (1024**3):.2f} GB"
    elif byte_value >= 1024**2:  # MB
        return f"{byte_value / (1024**2):.2f} MB"
    else:  # bytes
        return f"{byte_value} bytes"

def train_and_evaluate_blended_classifier(classifier_combo, additional_info, num_classifiers):
    print(f"Blending the following classifiers: {classifier_combo}")

    start_time = time.time()  # Record the start time
    start_time = time.time()
    # Get CPU information
    num_cores = multiprocessing.cpu_count()
    processor_type = platform.processor()
    # Get OS information
    os_name = platform.system()+'('+platform.release()+')'
    # Get RAM information
    ram_total = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')
    ram_total_mb = ram_total / (1024**2)
    # Calculate CPU and RAM resources before training
    cpu_before = psutil.cpu_percent()
    available_memory_before = psutil.virtual_memory().available

################################################################################
    # Create the voting classifier with the selected combination
    selected_classifiers = [(name, classifiers[name]) for name in classifier_combo]
    voting_classifier = VotingClassifier(estimators=selected_classifiers, voting='hard')

    # Train the voting classifier
    voting_classifier.fit(X_train_tfidf_dense, y_train)

    # Make final predictions on the test data using the voting classifier
    final_predictions = voting_classifier.predict(X_test_tfidf_dense)

    # Calculate evaluation metrics for the blended model
    accuracy = accuracy_score(y_test, final_predictions)
    precision = precision_score(y_test, final_predictions, average='weighted', zero_division=1)
    recall = recall_score(y_test, final_predictions, average='weighted')
    f1 = f1_score(y_test, final_predictions, average='weighted')
    cm = confusion_matrix(y_test, final_predictions)

    # Calculate Cohen's kappa
    kappa = cohen_kappa_score(y_test, final_predictions)

################################################################################

    end_time = time.time()  # Record the end time
    execution_time = end_time - start_time  # Calculate the execution time
    # Calculate CPU and RAM resources after training
    cpu_after = psutil.cpu_percent()
    available_memory_after = psutil.virtual_memory().available

    # Calculate resource usage during this iteration
    cpu_usage = max(cpu_after - cpu_before, 0)
    memory_usage = max(available_memory_after - available_memory_before, 0)  # Ensure non-negative value
    memory_usage_str = bytes_to_mb_or_gb(memory_usage)

    print(f"CPU Usage: {cpu_usage:.2f}%")
    print(f"Memory Usage: {memory_usage_str}")

    # Store the results in the list as a dictionary
    results_dict = {
        'Blended Classifiers': classifier_combo,
        'Accuracy': accuracy,
        'Kappa': kappa,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Confusion Matrix': cm,
        'Execution Time (s)': execution_time,
        'Total Classifiers': additional_info['Total Classifiers'],
        'Total Features': additional_info['Total Features'],
        'Training Data Size': additional_info['Training Data Size'],
        'Test Data Size': additional_info['Test Data Size'],
        'Random State': additional_info['Random State'],
        'Preprocessing': additional_info['Preprocessing'],
        'SMOTE': additional_info['SMOTE'],
        'Total CPU Cores': num_cores,
        'CPU Usage (%)': cpu_usage,
        'Total RAM': ram_total_mb,
        'Memory Usage': memory_usage_str,
        'Processor Type': processor_type,
        'OS': os_name
    }
    print(results_dict)


    return results_dict

Word2Vec:
We can use Word2Vec in our code to learn word embeddings from your text data. To do this, we'll need to use a library like Gensim in Python, which provides an easy way to train Word2Vec models.

Index(['Sentence', 'Sentiment', 'Preprocessed_Sentence'], dtype='object')


In [8]:
# Generating our own word2vec model

from gensim.models import Word2Vec, KeyedVectors
df['Preprocessed_Sentence'] = df['Sentence'].apply(preprocess_text)
# print(df.columns)
corpus = df['Preprocessed_Sentence'].apply(lambda x: x.split())  # Assuming each sentence is a space-separated list of words

# Train Word2Vec model with more epochs
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1, sg=0, epochs=10)

# Monitor Loss: You can track the loss (negative log likelihood) during training using the get_latest_training_loss() method of the Word2Vec model. This method returns the loss for the most recent batch of training data:
# Check the latest training loss
latest_loss = word2vec_model.get_latest_training_loss()
print(f"Latest Training Loss: {latest_loss}")
# The loss should decrease over time during training. If it remains constant or increases significantly, it may indicate an issue with your data or training parameters.

# Get the list of words in your Word2Vec model's vocabulary
vocabulary_words = word2vec_model.wv.index_to_key
# Print the first 10 words as an example
print("First 10 words in the vocabulary:")
print(vocabulary_words[:10])

# Test Word Similarity: After training, you can test whether the model has learned meaningful word embeddings by checking word similarity. Gensim's wv attribute provides access to the word vectors. You can use the similarity method to check the similarity between words:
# Check word similarity between two specific words
# word1 = "the"
# word2 = "and"
# similarity_score = word2vec_model.wv.similarity(word1, word2)
# print(f"Similarity between '{word1}' and '{word2}': {similarity_score}")
# Replace "word1" and "word2" with words from your vocabulary to check their similarity. Higher similarity scores indicate that the model has learned meaningful word representations.

# Save the model to a file
word2vec_model.save("word2vec_model.bin")

# Load the model from the file
loaded_model = Word2Vec.load("word2vec_model.bin")

loaded_model

# Save the trained model to a local file
# word2vec_model.save('word2vec_model.bin')

# import shutil

# # Specify the destination folder in your Google Drive
# destination_folder = '/content/Drive/MyDrive/python/financial_sentiment_analysis/'

# # Move the trained model to Google Drive
# shutil.move('word2vec_model.bin', destination_folder)

Latest Training Loss: 0.0
First 10 words in the vocabulary:
['.', 'the', ',', 'and', 'of', 'in', 'to', 'point', 'two', 'a']


In [13]:
# Vectorizing, Balancing, Extracting Features and Splitting the Data.

# Constants
PREPROCESSING_FLAG = 1
SMOTE_FLAG = 1
# MAX_FEATURES_LIMIT = 15000
TEST_SIZE = 0.2
RAND_STATE = 42

# Apply data preprocessing to the 'Sentence' column and create 'Preprocessed_Sentence' column
df['Preprocessed_Sentence'] = df['Sentence'].apply(preprocess_text)

# Prepare the data
if PREPROCESSING_FLAG == 1:
  X = df['Preprocessed_Sentence']
else:
  X = df['Sentence']

y = df['Sentiment']

################# TESTING #####################
# Initialize lists to store sentiment scores for each document
positive_scores = []
negative_scores = []
uncertainty_scores = []
litigious_scores = []
strong_modal_scores = []
weak_modal_scores = []
constraining_scores = []

# Iterate through your dataset (replace df with your actual DataFrame)
for text in df['Preprocessed_Sentence']:
    tokens = text.split()
    # print(tokens)

    # Initialize sentiment scores for each document
    sentiment_scores = {
        'positive': 0,
        'negative': 0,
        'uncertainty': 0,
        'litigious': 0,
        'strong_modal': 0,
        'weak_modal': 0,
        'constraining': 0
    }

    # Calculate sentiment scores for each token in the document
    for token in tokens:
        token = token.upper()  # Convert token to uppercase for matching
        if token in master_dictionary:
            entry = master_dictionary[token]
            for sentiment_category in sentiment_scores.keys():
                if getattr(entry, sentiment_category) > 0:
                    sentiment_scores[sentiment_category] += 1

    # Append sentiment scores to their respective lists
    positive_scores.append(sentiment_scores['positive'])
    negative_scores.append(sentiment_scores['negative'])
    uncertainty_scores.append(sentiment_scores['uncertainty'])
    litigious_scores.append(sentiment_scores['litigious'])
    strong_modal_scores.append(sentiment_scores['strong_modal'])
    weak_modal_scores.append(sentiment_scores['weak_modal'])
    constraining_scores.append(sentiment_scores['constraining'])

# Add sentiment score columns to your DataFrame
df['Positive_Score'] = positive_scores
df['Negative_Score'] = negative_scores
df['Uncertainty_Score'] = uncertainty_scores
df['Litigious_Score'] = litigious_scores
df['Strong_Modal_Score'] = strong_modal_scores
df['Weak_Modal_Score'] = weak_modal_scores
df['Constraining_Score'] = constraining_scores

# Print positive and negative scores for the first few documents (adjust the range as needed)
# for i in range(50):  # Print scores for the first 50 documents as an example
#     print(f"Document {i + 1} - Positive Score: {positive_scores[i]}, Negative Score: {negative_scores[i]}")

################# Feature Extraction #####################

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer() # max_features=MAX_FEATURES_LIMIT
X_train = tfidf_vectorizer.fit_transform(X)

# Convert the labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

loaded_model = Word2Vec.load("word2vec_model.bin")

# Iterate through the documents to create Word2Vec feature vectors
word2vec_features = []

for doc in df['Preprocessed_Sentence']:
    tokens = doc.split()
    word2vec_embeddings = []

    for token in tokens:
        if token in loaded_model.wv:
            word2vec_embeddings.append(loaded_model.wv[token])

    # Calculate the average Word2Vec embedding for the document
    avg_embedding = sum(word2vec_embeddings) / len(word2vec_embeddings) if word2vec_embeddings else [0.0] * loaded_model.vector_size

    # Convert the average Word2Vec embedding to a dense array
    avg_embedding_dense = [float(x) for x in avg_embedding]

    word2vec_features.append(avg_embedding_dense)

# Convert the list of Word2Vec features to a numpy array
X_word2vec = np.array(word2vec_features)

# Merge TF-IDF, sentiment scores, and Word2Vec features
from scipy.sparse import hstack

X_features = hstack((X_train, df[['Positive_Score', 'Negative_Score', 'Uncertainty_Score', 'Litigious_Score', 'Strong_Modal_Score', 'Weak_Modal_Score', 'Constraining_Score']].values), format='csr')
X_combined = hstack((X_features, X_word2vec), format='csr')

# Print the number of features added
print(f"Number of features added: {X_combined.shape[1]}")

# Balancing the Dataset
smote = SMOTE(sampling_strategy='auto', random_state=RAND_STATE) # random_state=RAND_STATE
X_train_balanced, y_train_balanced = smote.fit_resample(X_combined, y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_balanced, y_train_balanced, test_size=TEST_SIZE, random_state=RAND_STATE) # random_state=RAND_STATE

# Convert X_train_tfidf to a dense numpy array
X_train_tfidf_dense = X_train.toarray()

# Convert X_test_tfidf to a dense numpy array
X_test_tfidf_dense = X_test.toarray()

if np.any(X_train_tfidf_dense < 0):
    # Handle or debug the presence of negative values in TF-IDF features
    print("Negative values found in TF-IDF features.")

# X_train_tfidf_dense
# y_encoded


KeyboardInterrupt: ignored

In [10]:
individual_mode = False

# Initialize classifiers as base models
classifiers = {
    # 'Logistic Regression': LogisticRegression(max_iter=1000),
    'SDG': SGDClassifier(),
    # 'Random Forest': RandomForestClassifier(max_depth=10),
    'Support Vector Machine': SVC(),
    # 'Naive Bayes': MultinomialNB(),
    #  'Decision Tree': DecisionTreeClassifier(),
    #  'Bagging': BaggingClassifier(),
    # 'Gaussian Naive Bayes': GaussianNB(),
    # 'Extreme Gradient Boosting (XGBoost)': XGBClassifier()

}

# Initialize blended_results_list
results_list = []
# Initialize blended_results_dict
results_dict = {}
# Initialize an empty list to store the rows of the results
results_rows = []
# Additional Information
# Additional information about the code
additional_info = {
    'Total Classifiers': len(classifiers),
    'Total Features': {X_combined.shape[1]},
    # 'Total Features': len(tfidf_vectorizer.get_feature_names_out()),
    'Training Data Size': len(y_train),
    'Test Data Size': len(X_test_tfidf_dense),
    'Random State': RAND_STATE,  # Replace with the actual random_state value used in train_test_split
    'Preprocessing': PREPROCESSING_FLAG,
    'SMOTE': SMOTE_FLAG
}



In [11]:
# Evaluating

for r in range(2, len(classifiers) + 1):
    combinations_list = list(combinations(classifiers, r))

    # Iterate through each combination and call the train_and_evaluate_blended_classifier method
    for combo in combinations_list:
        num_classifiers = len(combo)  # Get the number of classifiers in the current combination
        results = train_and_evaluate_blended_classifier(combo, additional_info, num_classifiers)
        results_list.append(results)

# Sort the blended results based on accuracy in descending order
sorted_results = sorted(results_list, key=lambda x: (x['Accuracy'], x['Precision'], x['Recall'], x['F1-Score']), reverse=True)

# Save the sorted_results and additional information to a DataFrame
sorted_results_df = pd.DataFrame(sorted_results)
print(sorted_results_df)

# Set the time zone to Pakistan Standard Time (PST)
pakistan_tz = pytz.timezone('Asia/Karachi')

# Get the current date in Pakistan's time zone
current_date = datetime.now(pakistan_tz).strftime('%Y-%m-%d')

# Append the new results to the existing file or create a new file if it doesn't exist
file_name = f"lm_{current_date}_blended_classifier_results.csv"
file_path = f"/content/Drive/MyDrive/python/results/blended_classifiers/{file_name}"

try:
    if os.path.exists(file_path):
        existing_results_df = pd.read_csv(file_path)
        final_df = pd.concat([existing_results_df, sorted_results_df], ignore_index=True)
    else:
        final_df = sorted_results_df

    final_df.to_csv(file_path, index=False)
    print(f"Results have been appended to {file_path} in Google Drive.")
except Exception as e:
    print(f"An error occurred while saving the file: {str(e)}")


Blending the following classifiers: ('SDG', 'Support Vector Machine')
CPU Usage: 34.50%
Memory Usage: 0 bytes
{'Blended Classifiers': ('SDG', 'Support Vector Machine'), 'Accuracy': 0.7603833865814696, 'Kappa': 0.6400912228849344, 'Precision': 0.7798262949382323, 'Recall': 0.7603833865814696, 'F1-Score': 0.7569033531192514, 'Confusion Matrix': array([[587,  30,  14],
       [147, 456,  29],
       [ 97, 133, 385]]), 'Execution Time (s)': 795.9846560955048, 'Total Classifiers': 2, 'Total Features': {11199}, 'Training Data Size': 7512, 'Test Data Size': 1878, 'Random State': 42, 'Preprocessing': 1, 'SMOTE': 1, 'Total CPU Cores': 2, 'CPU Usage (%)': 34.49999999999999, 'Total RAM': 12982.625, 'Memory Usage': '0 bytes', 'Processor Type': 'x86_64', 'OS': 'Linux(5.15.120+)'}
             Blended Classifiers  Accuracy     Kappa  Precision    Recall  \
0  (SDG, Support Vector Machine)  0.760383  0.640091   0.779826  0.760383   

   F1-Score                                 Confusion Matrix  \
0  